# Heterogenous pure mechanics

We aim to solve
$$\nabla\cdot \sigma = -f$$
with 
$$\sigma = 2\mu \epsilon + \lambda \text{tr}(\epsilon) I$$
where $\epsilon = (\nabla u + (\nabla u)^T)/2$.

The additional feature is that $\mu = \mu(x,y)$ and $\lambda = \lambda (x,y)$. Physically, $\mu$ is the shear modulus (often denoted $G$). $\mu := \sigma_{xy}/\epsilon_{xy}$ (for relatively small strains), where $\sigma_{xy}$ is shear stress, $\epsilon_{xy}$ is shear strain. It measures the stiffness of materials - i.e. the material's response to shear stress. $\lambda$ on the other hand, may be difficult to interpret physically. 

In [ ]:
import porepy as pp
import numpy as np
import scipy.sparse as sps

## Insert a heterogenity using a rectangle method 

In [ ]:
class Rectangle:
    
    def __init__(self, physdims, foot=np.array([0,0])):
        """ Create a rectangle represented by its vertices in CCW order.
        
        Parameters
        physdims (np.array, (2,) ): Physical dimensions of rectangle
        foot (np.array, (2,) ): Coordinates of SW foot of rectangle. Default: (0,0)
        """
        assert(np.all(np.array(physdims) > 0))
        ft = np.array([foot]).T
        hz = np.array([[physdims[0], 0]]).T # Horizontal shift
        vt = np.array([[0, physdims[1]]]).T # Vertical shift
        self.L = np.hstack((ft, ft+hz))
        self.U = np.add(np.flip(self.L, axis=1), vt)
        self.nodes = np.hstack((self.L, self.U))
        
        # Construction assertion:
        #from porepy.geometry.geometry_property_checks import is_ccw_polygon
        #assert(is_ccw_polygon(self.nodes))
        
        
class RectangleByCenter(Rectangle):
    
    def __init__(self, physdims, center):
        """ Create a rectangle from a center coordinate.
        
        See rectangle for more information.
        
        Parameters
        physdims (np.array, (2,) ): Physical dimensions of rectangle
        foot (np.array, (2,) ): Coordinates of center of rectangle
        
        """
        c = np.array(center); pd = np.array(physdims)
        
        foot = c - pd/2
        super().__init__(physdims, foot)

## The mechanics simulation class 

In [ ]:
class Mechanics:
    """ Class for setting up a pure mechanics problem with no fractures.
    
    The following attributes are assigned to self:
    Method: __init__
        mesh_args (dict): Mesh sizes
        folder_name (str): Name of folder for visualization
        displacement_variable (str): Name of variable for displacement
        mechanics_parameter_keyword (str): Parameter keyword for mechanics problems
    Method: create_grid
        box (dict): Bounding box of domain.
        gb (pp.GridBucket): The grid bucket.
        Nd (int): Dimension of the domain.
    Method: rock_parameters
        lam (np.ndarray): lambda, Lamé parameter
        mu (np.ndarray): Lamé parameter
        
    The following attributes in self may be updated:
    Method: insert_heterogenity
        lam (np.ndarray)
        mu (np.ndarray)
        
    """

    def __init__(self, mesh_args, folder_name):
        """ Initialize the mechanics class.
        
        Parameters
        mesh_args (dict): Containing mesh sizes.
                For now, this should be e.g. {'n': 5}. 
                We use cartesian meshes.
        folder_name (str): Name of storage folder for visualization.
        
        """
        from porepy.geometry.geometry_property_checks import point_in_polygon
        self.mesh_args = mesh_args
        self.folder_name = folder_name
        
        # Variable
        self.displacement_variable = "u"
        
        # Keyword
        self.mechanics_parameter_keyword = "mechanics"
        
        
    def create_grid(self):
        """ Create a 2D domain without fractures.
        
        The method requires the following attribute:
            mesh_args (dict): Containing the mesh sizes
        
        The following attributes are assigned to self:
            box (dict): Bounding box of domain.
            gb (pp.GridBucket): The grid bucket.
            Nd (int): Dimension of the domain.
        
        """
        n = self.mesh_args['n']
        nx = np.array((n,n))
        self.box = {'xmin': 0, 'ymin': 0, 'xmax': 1, 'ymax': 1}
        
        #g = pp.CartGrid([n,n], physdims=[1,1])
        #g.compute_geometry()
        gb = pp.meshing.cart_grid([], nx, physdims=[1,1])
        gb.compute_geometry()
        gb.assign_node_ordering()
        
        self.gb = gb
        self.Nd = gb.dim_max()
        
    def domain_boundary_sides(self, g):
        """ 
        Obtain indices for faces on each side of the domain boundary.
        
        """
        tol = 1e-10
        box = self.box
        east = g.face_centers[0] > box["xmax"] - tol
        west = g.face_centers[0] < box["xmin"] + tol
        north = g.face_centers[1] > box["ymax"] - tol
        south = g.face_centers[1] < box["ymin"] + tol
        if self.Nd == 2:
            top = np.zeros(g.num_faces, dtype=bool)
            bottom = top.copy()
        else:
            top = g.face_centers[2] > box["zmax"] - tol
            bottom = g.face_centers[2] < box["zmin"] + tol
        
        all_bf = g.get_boundary_faces()
        return all_bf, east, west, north, south
        
    def bc_type(self, g):
        all_bf, *_ = self.domain_boundary_sides(g)
        bc = pp.BoundaryConditionVectorial(g, all_bf, "dir")
        return bc
    
    def bc_values(self, g):
        values = np.zeros((self.Nd, g.num_faces))
        values = values.ravel('F')
        return values

    def source(self, g):
        return np.zeros(g.dim * g.num_cells)
    
    def rock_parameters(self, g):
        """ Assemble the stress-strain tensor.
        
        The following attributes are assigned to self:
            lam (np.ndarray, (g.num_cells,)): lambda parameter
            mu (np.ndarray, (g.num_cells,)): mu parameter.

        """
        lam = np.ones(g.num_cells)
        mu = np.ones(g.num_cells)
        C = pp.FourthOrderTensor(mu, lam)
        self.lam = lam
        self.mu = mu
        return C
    
    def set_parameters(self):
        """ 
        Set default parameters to the matrix

        """
        for g, d in self.gb:
            if g.dim == self.Nd:
                
                # Rock parameters
                C = self.rock_parameters(g)
                
                # Boundary conditions and source value
                bc = self.bc_type(g)
                bc_val = self.bc_values(g)
                f = self.source(g)

                specified_parameters = {'fourth_order_tensor': C, 'source': f, 
                                        'bc': bc, 'bc_values': bc_val}
                pp.initialize_default_data(g, d, self.mechanics_parameter_keyword, 
                                           specified_parameters)
        
    def insert_heterogenity(self, physdims, center, mu, lam, tol=1e-8):
        """ Insert a heterogenity to the domain.

        A heterogenity in this sense means a rectangle with
        a different mu and/or lambda (e.g. different C) than
        the surrounding matrix.
        Assumes that parameters are already been set. Assumes existense
        of attributes self.lam and self.mu

        Parameters:
        physdims (np.ndarray, (2,) ): Physical dimensions of rectangle
        center (np.ndarray, (2,)): Coordinates of center of rectangle
        mu (float): Value of mu in the rectangle.
        lam (float): Value of lambda in the rectangle.
        tol (float, Default: 1e-8): Tolerance of rectangle bounds.

        The following attributes in self are updated:
            mu (np.ndarray, (g.num_cells,)): mu parameter
            lam (np.ndarray, (g.num_cells,)): Lambda parameter

        In addition, the parameter dictionary for the domain of highest dimension
        (we assume no fracs) has its entry 'fourth_order_tensor' updated

        """
        from porepy.geometry.geometry_property_checks import point_in_polygon
        g = self.gb.grids_of_dimension(self.Nd)[0]
        data = self.gb.node_props(g)
        R = RectangleByCenter(physdims, center)

        # This code assumes 2D.
        nds = np.copy(g.cell_centers)
        nds[2] = np.array(range(g.num_cells)) # Use z-coordinate as index.
        rect_nodes = nds[2, point_in_polygon(R.nodes, nds[[0,1],:], tol=tol)].astype(int)

        self.mu[rect_nodes] = mu
        self.lam[rect_nodes] = lam
        C = pp.FourthOrderTensor(self.mu, self.lam)

        data[pp.PARAMETERS]['fourth_order_tensor'] = C
        
    def assign_variables(self):
        """ 
        Assign variables to the node of the grid bucket.

        """
        gb = self.gb
        for g, d in gb:
            if g.dim == self.Nd:
                d[pp.PRIMARY_VARIABLES] = {
                    self.displacement_variable: {"cells": self.Nd}
                }
            else:
                d[pp.PRIMARY_VARIABLES] = {}
        
        for e, d in gb.edges():
            
            if e[0].dim == self.Nd:
                pass
            
            d[pp.PRIMARY_VARIABLES] = {}
    
    def assign_discretization(self):
        """ 
        Assign discretization to the node of the grid bucket
        
        """
        # We solve the highest dimension problem using mpsa.
        # There should be no other dimensions present.
        Nd = self.Nd
        gb = self.gb
        mpsa = pp.Mpsa(self.mechanics_parameter_keyword)
        
        for g, d in gb:
            if g.dim == Nd:
                d[pp.DISCRETIZATION] = {
                    self.displacement_variable: {'mpsa': mpsa}}
        
        # Assume no edges, so we don't loop over them.
        

## A mechanics subclass
Subclass with different specific implementations (different boundary conditions)

In [ ]:
class MechanicsBCExample1(Mechanics):
    """ This subclass sets up the pure mechanics problem where we have
    0 dirichlet everywhere except on the top boundary, with neumann
    pushing down."""
    
    def _bc_helper(self, g):
        """ Local helper function for bc_type and bc_values.
        Gets the indices we need.
        
        Parameters
        g (pp.Grid): A grid.
        """
        all_bf, east, west, north, south = self.domain_boundary_sides(g)
        dir_sides = np.logical_or(np.logical_or(west, south), east)
        dir_sides = np.nonzero(dir_sides)[0]
        neu_sides = np.nonzero(north)[0]
        return dir_sides, neu_sides
        
        
    
    def bc_type(self, g):
        # Boundary condition setup 1:
        # 0 dirichlet everywhere. Neumann push down on top.
        dir_sides, neu_sides = self._bc_helper(g)
        bc = pp.BoundaryConditionVectorial(g, dir_sides, ['dir']*dir_sides.size)
        return bc
        
        
    def bc_values(self, g):
        dir_sides, neu_sides = self._bc_helper(g)
        u_b = np.zeros((g.dim, g.num_faces))
        u_b[1, neu_sides] = -1 * g.face_areas[neu_sides]
        u_b[:, dir_sides] = 0
        return u_b.ravel('F')

## Method to set up and run a mechanics model

In [ ]:
def run_mechanics(setup, file_name=None):
    """
    Method for solving linear elasticity on a non-fractured (currently 2D) domain.
    
    Arguments:
        setup: A setup class with methods:
                create_grid(): Create and return the grid bucket
                set_parameters(): assigns data to grid bucket.
                assign_variables(): assigns variables on grid bucket nodes and edges.
                assign_discretizations(): assigns discretizations on grid bucket nodes
                    and edges.
            and attributes:
                folder_name: returns a string. The data from the simulation will be
                written to the file 'folder_name/' + setup.out_name and the vtk files to
                'res_plot/' + setup.out_name
        file_name (str, Optional): File name for visualization of simulation result. 

    """
    # File name
    if file_name is None:
        file_name = 'mechanics'
    
    # Define the grid without overwriting an existing one.
    if "gb" in setup.__dict__:
        gb = setup.gb
    else:
        gb = setup.create_grid()
        gb = setup.gb
    
    # Set parameters, variables and discretizations on grid.
    setup.set_parameters()
    setup.assign_variables()
    setup.assign_discretization()
    
    # Setup assembler and discretize
    assembler = pp.Assembler(gb)
    assembler.discretize()
    
    
    g = gb.grids_of_dimension(2)[0]
    data = gb.node_props(g)
    viz = pp.Exporter(g, name=file_name, folder=setup.folder_name)
    
    # Solve the system
    A, b = assembler.assemble_matrix_rhs()
    solution = sps.linalg.spsolve(A, b)
    assembler.distribute_variable(solution)
    
    u2 = data[pp.STATE][setup.displacement_variable]
    viz.write_vtk({"ux": u2[::2], "uy": u2[1::2]})
    
    return u2

## Trivial test case 

In [ ]:
mesh_args = {'n': 5}
folder_name = "heterogenous_test"
file_name = 'trivial_case1'

mech = Mechanics(mesh_args, folder_name)

u2 = run_mechanics(mech, file_name=file_name)

# Non-trivial test case 1
Uses the sub-method MechanicsBCExample1

In [ ]:
mesh_args = {'n': 10}
folder_name = "heterogenous_nontrivial_test"
file_name = 'non-het__mu-1'

mech = MechanicsBCExample1(mesh_args, folder_name)

u2 = run_mechanics(mech, file_name=file_name)

# Heterogeneous test case
We want to add a (rectangular) region to the center of the domain with *weaker* material properties.

* How do we ensure that the material is weaker?
- Let's consider the terms of the stress-strain relation:
\begin{align}
\sigma = {} & 2\mu \epsilon + \lambda \text{tr}(\epsilon) I \\
= {} & \mu (\nabla u + (\nabla u)^T) + \lambda (v_{x} + w_{y}) I
\end{align}
since $\epsilon = (\nabla u + (\nabla u)^T)/2$. Notationally, we let
\begin{equation}
u =
\begin{bmatrix}
v \\ w
\end{bmatrix}
, \quad \implies
\nabla u = 
\begin{bmatrix}
v_x & v_y \\
w_x & w_y \\
\end{bmatrix}
\end{equation}
where $v$ is the displacement of $u$ in the x-direction, and $w$ in the y-direction.

### Looking closer at $\mu$
Consider the approximate relation for the shear modulus,
\begin{equation}
\mu := \frac{\sigma_{xy}}{\epsilon_{xy}}
\end{equation}
We may write it as $\mu\epsilon_{xy} = \sigma_{xy}$. This means that for a constant shear stress $\sigma_{xy}$, if $\mu$ is high, then $\epsilon_{xy}$ will be low. I.e., less shear strain is induced with a high shear modulus. \
So, if we aim to design a "weaker" region, we lower the shear modulus. Let $\lambda$ remain constant and equal everywhere.

In [ ]:
class HeterogenousMechanics(Mechanics):
    """ 
    Subclass of the non-trivial test case 1.
    
    Will overwrite the method 'set_parameters' to 
    introduce a heterogenity.
    
    """
    
    def __init__(self, het_args, *args):
        """ 
        Assign specific mu and lambda parameters in the heterogenous region.
        
        Parameters
        het_args (dict): Contains the following keys:
                mu_het (float): Parameter mu
                lam_het (float): Parameter lambda
                rel_size (np.array, (2,)): Width and heigh of heterogenous region, as
                        fraction of the total region in that respective dimension.
        kwargs: Arguments for parent class.

        """
        super().__init__(*args)

        self.lam_het = het_args.get('lam_het', 1)
        self.mu_het = het_args.get('mu_het', 1)
        self.rel_size = het_args.get('rel_size', np.array([0.2, 0.2]))
    
    def set_parameters(self):
        """ 
        Set default parameters to the matrix
        OVERRIDES SUPER METHOD.

        """
        # First run the parent method
        super().set_parameters()
        
        # Location of heterogenity
        bbox = self.box
        het_x = (bbox['xmax']-bbox['xmin']) * self.rel_size[0]
        het_y = (bbox['ymax']-bbox['ymin']) * self.rel_size[1]
        physdims = np.array([het_x, het_y])
        
        center = np.array([bbox['xmax']+bbox['xmin'], bbox['ymax']+bbox['ymin']]) / 2

        # Insert heterogenity to region with the desired parameters.
        self.insert_heterogenity(physdims, center, self.mu_het, self.lam_het)
    
    
    def _bc_helper(self, g):
        """ Local helper function for bc_type and bc_values.
        Gets the indices we need.
        
        Parameters
        g (pp.Grid): A grid.
        """
        all_bf, east, west, north, south = self.domain_boundary_sides(g)
        dir_sides = np.logical_or(np.logical_or(west, south), east)
        dir_sides = np.nonzero(dir_sides)[0]
        neu_sides = np.nonzero(north)[0]
        return dir_sides, neu_sides
    
    def bc_type(self, g):
        # Boundary condition setup 1:
        # 0 dirichlet everywhere. Neumann push down on top.
        dir_sides, neu_sides = self._bc_helper(g)
        bc = pp.BoundaryConditionVectorial(g, dir_sides, ['dir']*dir_sides.size)
        return bc
        
    def bc_values(self, g):
        dir_sides, neu_sides = self._bc_helper(g)
        u_b = np.zeros((g.dim, g.num_faces))
        u_b[1, neu_sides] = -1 * g.face_areas[neu_sides]
        u_b[:, dir_sides] = 0
        return u_b.ravel('F')
        

In [ ]:
mesh_args = {'n': 10}
folder_name = "heterogenous_nontrivial_test"
file_name = "het__mu-0.5"
het_args = {'lam_het': 1,
            'mu_het': 0.5,
            'rel_size': np.array([0.2, 0.2])}

mech = HeterogenousMechanics(het_args, mesh_args, folder_name)

u2 = run_mechanics(mech, file_name=file_name)

### Scale $\mu$ down by another half
I.e., let $\mu = 0.25$.

In [ ]:
file_name = 'het__mu-0.25'
het_args['mu_het'] = 0.25

mech = HeterogenousMechanics(het_args, mesh_args, folder_name)

u2 = run_mechanics(mech, file_name=file_name)

# Heterogenous Mechanics with BC as in MPSA-intro 
Due to the difficulty in interpreting the results in the above simulations, I compute the simulations with same BC as in the introductory example in `MPSA-intro`, which hopefully illuminates some questions.

In [ ]:
class HeterogenousMechanicsIntroBC(HeterogenousMechanics):
    """ Set a heterogenous mechanics problem (a weakness in the middle part of the domain),
    but with 'more familiar' boundary conditions, as in the MPSA-intro notebook.
    """
    
    def _bc_helper(self, g):
        """ Local helper function for bc_type and bc_values.
        Gets the indices we need.
        
        Set dirichlet sides as south side.
        Neumann sides are the rest.
        Specifically, for the bc_values method - return neu_sides as north side
        to set a downward pushing force on the top boundary.
        
        Parameters
        g (pp.Grid): A grid.
        """
        all_bf, east, west, north, south = self.domain_boundary_sides(g)
        dir_sides = np.nonzero(south)[0]
        neu_sides = np.nonzero(north)[0]
        return dir_sides, neu_sides
    
    # Note: The methods themselves for boundary conditions and boundary values are exactly those
    # in HeterogenousMechanics. The difference is which sides we tell that method is the dirichlet
    # and neumann sides.

## 1. Base case: $\mu = 1$ (i.e.: Homogenous domain)
This is exactly the same as `MPSA-intro` notebook.

In [ ]:
folder_name = "MPSA-intro-heterogenous"

In [ ]:
file_name = "mpsa-intro-het__mu-1"

mesh_args = {'n': 10}
het_args = {'lam_het': 1,
            'mu_het': 1,
            'rel_size': np.array([0.2, 0.2])}

mech = HeterogenousMechanicsIntroBC(het_args, mesh_args, folder_name)

u2 = run_mechanics(mech, file_name=file_name)

## 2: Scale by a half - i.e. $\mu = 0.5$ in the heterogenous region 

In [ ]:
file_name = "mpsa-intro-het__mu-0.5"

mesh_args = {'n': 10}
het_args = {'lam_het': 1,
            'mu_het': 0.5,
            'rel_size': np.array([0.2, 0.2])}

mech = HeterogenousMechanicsIntroBC(het_args, mesh_args, folder_name)

u2 = run_mechanics(mech, file_name=file_name)

## 3: Scale by a fourth - i.e. $\mu = 0.25$ in the heterogenous region 

In [ ]:
## Second: Scale by a half - i.e. $\mu = 0.5$ in the heterogenous region 

file_name = "mpsa-intro-het__mu-0.25"

mesh_args = {'n': 10}
het_args = {'lam_het': 1,
            'mu_het': 0.25,
            'rel_size': np.array([0.2, 0.2])}

mech = HeterogenousMechanicsIntroBC(het_args, mesh_args, folder_name)

u2 = run_mechanics(mech, file_name=file_name)

## 4: Scale by an eight - i.e. $\mu = 0.125$ in the heterogenous region 

In [ ]:
## Second: Scale by a half - i.e. $\mu = 0.5$ in the heterogenous region 

file_name = "mpsa-intro-het__mu-0.125"

mesh_args = {'n': 10}
het_args = {'lam_het': 1,
            'mu_het': 0.125,
            'rel_size': np.array([0.2, 0.2])}

mech = HeterogenousMechanicsIntroBC(het_args, mesh_args, folder_name)

u2 = run_mechanics(mech, file_name=file_name)